In [1]:
%load_ext dotenv
%dotenv

## Import Dependencies

In [2]:
import os

import librosa

# Imports for IBM Watson's Speech to Text Transcriber
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Segmentation of the Audiobook file into 30 sec clips

## Open Audiobook File

In [3]:
file_name = r'..\\..\\..\\assets\\audio_files\\Alice_in_Wonderland_chap_1.mp3'
audio, sr = librosa.load(file_name)

c:\users\danny\miniconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


## Split and Save audio files

In [4]:
save_path = r'..\\..\\..\\assets\\audio_files\\temp'
 
buffer = 30 * sr
 
samples_total = len(audio)
samples_wrote = 0
counter = 1
 
while samples_wrote < samples_total:
  
    #check if the buffer is not exceeding total samples 
    if buffer > (samples_total - samples_wrote):
        buffer = samples_total - samples_wrote
 
    block = audio[samples_wrote : (samples_wrote + buffer)]
    out_file_name = "clip_" + str(counter) + ".wav"
    complete_name = os.path.join(save_path, out_file_name)
 
    librosa.output.write_wav(complete_name, block, sr)
    
    counter += 1
    samples_wrote += buffer

# Transcription of the audio clips and detection of emotions from the transcripts

## Setup STT Service

In [5]:
api_key = os.getenv("api_key")
url = os.getenv("url")

In [6]:
# Setup Service
authenticator = IAMAuthenticator(api_key)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

## Open Audio Source and Transcribe

In [8]:
# Enter the absolute path to the temp folder containing the clips
folder_path = r'D:\\Projects\\BEProject\\deep-audiobook-tuner\\assets\\audio_files\\temp'

file_names = os.listdir(folder_path)

# Don't run this loop with alll the clip files in the temp folder
for file_name in file_names:
    
    print(f"\nTranscription of {file_name}\n")
    
    file = os.path.abspath(os.path.join(folder_path, file_name))
    
    # Perform transcription
    with open(file, 'rb') as f:
        res = stt.recognize(audio=f, content_type='audio/wav', model='en-US_NarrowbandModel', continuous=True).get_result()
    
    text = ""
    for i in range(len(res['results'])):
        text += res['results'][i]['alternatives'][0]['transcript'][0:-1] + ". "
        
    print(text)
        
    # Print each transcribed line with its confidence
    conf = 0.0
    for i in range(len(res['results'])):
#         print(f"Transcript: {res['results'][i]['alternatives'][0]['transcript'][0:-1]}\n")
#         print(f"Confidence: {res['results'][i]['alternatives'][0]['confidence']}\n")
        conf += res['results'][i]['alternatives'][0]['confidence']
    print(f"\nConfidence: {conf/len(res['results'])}")
            
    print(f"\nDetected Emotions of {file_name}\n")
    emotions = te.get_emotion(text)
    print(emotions)
    
    print("\n---------------------------------------------------------------------------------------------------------------")


Transcription of clip_2.wav

good for the hot day made her feel very sleepy and stupid what are the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her there was nothing so very remarkable and that nor did Alice think it so very much out of the way to hear the rabbit seder itself oh dear oh dear I shall be late when she started over afterwards it occurred to her that she ought to have wondered if this. 
Confidence: 0.91

Detected Emotions of clip_2.wav

{'Happy': 0.29, 'Angry': 0.0, 'Surprise': 0.24, 'Sad': 0.29, 'Fear': 0.18}

---------------------------------------------------------------------------------------------------------------

Transcription of clip_3.wav

at the time it all seems quite natural but when the rabbit actually took a watch out of its waist coat pocket and looked at it and then hurried on ours started to her feet for it flashed across her mind that she had 